<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies_noncovid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 81kB 4.2MB/s 
     |████████████████████████████████| 122kB 18.4MB/s 


In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 11


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210411 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210411 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [19]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [20]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [22]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [23]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v_nc > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v_nc > 0) & \
    (df_cap.icu_o_nc > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 
                              'icu_v_nc', 'nonicu_v_nc', 'mechvent_v_nc', 
                              'icu_o_nc', 'nonicu_o_nc', 'mechvent_o_nc',
                              'Landline Number', 'Landline Number 2', 'Fax Number', 
                              'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v_nc', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [24]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,360071,2021-04-08,EAST AVENUE MEDICAL CENTER,QUEZON CITY,64,59,22,33,230,16,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,271043,2021-04-08,PHILIPPINE HEART CENTER,QUEZON CITY,48,36,52,9,43,14,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,278625,2021-04-08,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,45,175,18,17,106,15,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,109888,2021-04-08,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,40,40,4,13,57,2,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,257990,2021-04-08,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,34,141,14,15,58,5,(02) 8942 0245,None,None,Government
5,289894,2021-04-08,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,29,44,7,6,55,2,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
6,107628,2021-04-08,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,26,94,6,8,33,2,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
7,60224,2021-04-08,MANILA DOCTORS HOSPITAL,ERMITA,24,85,7,8,58,7,(02) 8558 0888,None,(02) 8524 7376,Private
8,52032,2021-04-08,ALLIED CARE EXPERTS (ACE) MEDICAL CENTER-QUEZO...,QUEZON CITY,24,0,2,6,0,0,(02) 8325 0725,(02) 8283 3993,(02) 8283 4003,Private
9,267372,2021-04-08,PASAY CITY GENERAL HOSPITAL,PASAY CITY,24,42,9,10,31,0,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government


In [25]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,360072,2021-04-09,EAST AVENUE MEDICAL CENTER,QUEZON CITY,63,91,22,35,211,17,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,271044,2021-04-09,PHILIPPINE HEART CENTER,QUEZON CITY,50,32,63,7,47,3,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,278626,2021-04-09,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,46,176,18,16,105,20,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,109889,2021-04-09,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,40,44,2,13,53,4,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,257991,2021-04-09,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,31,137,10,18,62,9,(02) 8942 0245,None,None,Government
5,289895,2021-04-09,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,30,31,5,5,68,2,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
6,107629,2021-04-09,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,28,103,8,6,24,0,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
7,267373,2021-04-09,PASAY CITY GENERAL HOSPITAL,PASAY CITY,25,50,9,9,23,0,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
8,160687,2021-04-09,"CAPITOL MEDICAL CENTER, INC.",QUEZON CITY,23,41,11,3,38,3,(02) 8372 3825,(02) 8938 7789,(02) 3411 4320,Private
9,60225,2021-04-09,MANILA DOCTORS HOSPITAL,ERMITA,23,90,7,9,53,7,(02) 8558 0888,None,(02) 8524 7376,Private


In [26]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v_nc,nonicu_v_nc,mechvent_v_nc,icu_o_nc,nonicu_o_nc,mechvent_o_nc,landline,landline2,fax,ownership
0,360073,2021-04-10,EAST AVENUE MEDICAL CENTER,QUEZON CITY,68,96,22,34,199,17,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
1,271045,2021-04-10,PHILIPPINE HEART CENTER,QUEZON CITY,51,38,55,5,41,11,(02) 8925 2401,(02) 8922 0551,(02) 8922 0551,Government
2,278627,2021-04-10,VETERANS MEMORIAL MEDICAL CENTER,QUEZON CITY,46,179,18,16,102,23,(02) 8920 2487,(02) 8927 1250,(02) 8926 2437,Government
3,109890,2021-04-10,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,39,28,2,14,69,4,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,257992,2021-04-10,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,31,146,10,18,53,9,(02) 8942 0245,None,None,Government
5,289896,2021-04-10,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,30,47,5,5,52,2,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
6,107630,2021-04-10,OUR LADY OF LOURDES HOSPITAL,CITY OF MANILA,29,106,8,5,21,0,(02) 8716 8001,(02) 8252 3942,(02) 8714 7153,Private
7,267374,2021-04-10,PASAY CITY GENERAL HOSPITAL,PASAY CITY,23,53,6,11,20,3,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government
8,160688,2021-04-10,"CAPITOL MEDICAL CENTER, INC.",QUEZON CITY,22,41,11,4,38,3,(02) 8372 3825,(02) 8938 7789,(02) 3411 4320,Private
9,60226,2021-04-10,MANILA DOCTORS HOSPITAL,ERMITA,22,93,7,10,50,7,(02) 8558 0888,None,(02) 8524 7376,Private


In [27]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o_nc'].sum()
vacant = df_cap[filter]['icu_v_nc'].sum()
pct_used = (occupied / (vacant + occupied)) * 100